In [29]:
import logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger("crewai.ipynb")

logger.info("logger is set up and ready to rock")

INFO:crewai.ipynb:logger is set up and ready to rock


# Crew Ai Sandbox

## What is Crew AI?

CrewAi is a framework to make it easier to work with roleplaying AI (LLM) agents to achieve a tree of thought. It is open source and its source code may be found on [GitHub](https://github.com/crewAIInc/crewAI)

## Concepts involved

* LLM (Large Language Models)
* Prompt Engineering

## Setting up

I recommend using `virtualenv`, once you have it create a new `virtualenv` on this folder, active it (by `source`ing on Unix systems the `activate` script) and restore dependencies by running `pip install -r requirements.txt`.

You'll also want to have `ollama` running locally in order to pull and use local LLMs. In order to save money this sandbox uses local environments instead of expensive OpenAI calls. Ollama can be found at [their official website](https://ollama.com/download).

To make it easier to interact with ollama without doing manual steps in the terminal or gui we'll be using `ollama-python` to set things up using the notebook itself.

### Problems with Python 3.13

If you are using Python 3.13 you might run into issues (as of 2024-12-14) when installing CrewAi thru pip.

In order to get my environment to work I had to:

1. Set up my machine to run `rust` by using the script from [RustUp.rs](https://rustup.rs)
2. Add rust's cargo tools to my path by sourcing it on `.zshrc`
3. Exporting a specific variable in order to toggle compatibility between 3.12 and 3.13 when compiling: `export PYO3_USE_ABI3_FORWARD_COMPATIBILITY=1`

## Experimenting

### Ensuring that ollama is available in the environment

Before committing to a solution I would like to ensure that it is possible to have things set in a way that they are repeatable so, I'll begin this experiment by looking at the `ollama` library and if (using it) we can tell whenever ollama is or isn't running.

From looking at their docs seems like the simplest way to ensure ollama is available is to try and `list` models as, even if there are no models available, the result is non-error. That way if there is an exception it means that somehow `ollama` either isn't available or isn't set up properly.

In [30]:
logger.info("ensuring that ollama is up and running")
from ollama import list

try:
    list()
    logger.info("ollama is up and running")
except:
    logger.error("ollama is not running, exiting")
    exit(1)


INFO:crewai.ipynb:ensuring that ollama is up and running
INFO:httpx:HTTP Request: GET http://127.0.0.1:11434/api/tags "HTTP/1.1 200 OK"
INFO:crewai.ipynb:ollama is up and running


Once we know it's available we can go ahead and pull from one model we know it's available. As of the time of this experiment the model I'm most aware of is `llama3` so I'll use it as the desired one.

An extensive list of available models may be found on [ollama's website](https://ollama.com/library).

Once a model is pulled and available locally we can prompt it, without creating a chat, by using the `generate` function and passing in a prompt we`d like to ask it.

In [31]:
from ollama import pull, show, generate
DESIRED_MODEL = "llama3"

# attempt to show the model, if it doesn't exist it means we need to pull it
try:
    show(DESIRED_MODEL)
    logger.info(f"model {DESIRED_MODEL} found locally, no need to pull")
except:
    logger.info(f"model {DESIRED_MODEL} not found, pulling it")
    pull(DESIRED_MODEL)
    logger.info(f"model {DESIRED_MODEL} pulled successfully")

logger.info("asking the model why the sky is blue")
response = generate(DESIRED_MODEL, "why is the sky blue, in up to 256 characters")
logger.info(f"model's response: {response.response}")


INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/show "HTTP/1.1 200 OK"
INFO:crewai.ipynb:model llama3 found locally, no need to pull
INFO:crewai.ipynb:asking the model why the sky is blue
INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/generate "HTTP/1.1 200 OK"
INFO:crewai.ipynb:model's response: The sky appears blue because of a phenomenon called Rayleigh scattering. Short-wavelength blue light is scattered more by tiny molecules of gases like nitrogen and oxygen, making it reach our eyes from all directions. This scattering effect dominates over other wavelengths, giving us the blue color!
